In [1]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [3]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [1]:
# Generate summary statistics
df_market_data.describe()

In [2]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [4]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
cc_scaled = StandardScaler().fit_transform(df_market_data)
cc_scaled

In [5]:
# Create a DataFrame with the scaled data
cc_scaled_df = pd.DataFrame(cc_scaled, columns = ['price_change_percentage_24h',
                                          'price_change_percentage_7d',
                                          'price_change_percentage_14d',
                                          'price_change_percentage_30d',
                                          'price_change_percentage_60d',
                                          'price_change_percentage_200d',
                                          'price_change_percentage_1y'                                          
                                         ])
cc_scaled_df.head()

In [7]:
# Copy the crypto names from the original data
coin_id = list(df_market_data.index)
coin_id

In [6]:
# Set the coinid column as index - the new column to the DataFrame
cc_scaled_df ['coin_id'] = coin_id
cc_scaled_df = cc_scaled_df.set_index('coin_id')

# Display sample data
cc_scaled_df.head()

---

### Find the Best Value for k Using the Original Data.

In [8]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 12))
k

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [9]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
for i in k:
    
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
    kmeans_model = KMeans(n_clusters = i, random_state = 0)
    
# 2. Fit the model to the data using `df_market_data_scaled`
    kmeans_model.fit(cc_scaled_df)
    
# 3. Append the model.inertia_ to the inertia list
    inertia.append(kmeans_model.inertia_)

In [10]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {'k': k, 'inertia': inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data)
elbow_df.head()

In [13]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
optimal_k_value = elbow_df.hvplot.line(x = 'k', y = 'inertia', title = 'Elbow Curve', xticks = k)
optimal_k_value

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [14]:
# Initialise the K-Means model using the best value for k
kmeans_model = KMeans(n_clusters = 4, random_state = 0)

In [11]:
# Fit the K-Means model using the scaled data
kmeans_model.fit(cc_scaled_df)

In [12]:
# Predict the clusters to group the cryptocurrencies using the scaled data
predict = kmeans_model.predict(cc_scaled_df)

# Print the resulting array of cluster values.
predict

In [17]:
# Create a copy of the DataFrame
cc_scaled_df_copy = cc_scaled_df.copy()

In [13]:
# Add a new column to the DataFrame with the predicted clusters
cc_scaled_df_copy['clusters'] = predict

# Display sample data
cc_scaled_df_copy.head()

In [19]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Colour the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
scatterplot = cc_scaled_df_copy.hvplot.scatter(x = 'price_change_percentage_24h',
                                               y = 'price_change_percentage_7d',
                                               hover_cols = 'coin_id',
                                               title = 'Original Scaled Data')
scatterplot

:Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimise Clusters with Principal Component Analysis.

In [20]:
# Create a PCA model instance and set `n_components=3`.
pca_model = PCA(n_components = 3)

In [14]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
pca_data = pca_model.fit_transform(cc_scaled_df)

# View the first five rows of the DataFrame. 
pca_data[:5]

In [15]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca_model.explained_variance_ratio_

In [16]:
# Summin the explained variance together to get the percentage - for reference
sum(pca_model.explained_variance_ratio_)

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 89%

In [17]:
# Create a new DataFrame with the PCA data.
df_market_data_pca = pd.DataFrame(pca_data, columns = ['PCA1', 'PCA2', 'PCA3'])

# Creating a DataFrame with the PCA data

# Copy the crypto names from the original data
coin_id = cc_scaled_df.index

# Set the coinid column as index
df_market_data_pca['coin_id'] = coin_id
df_market_data_pca = df_market_data_pca.set_index('coin_id', drop = True)

# Display sample data
df_market_data_pca.head()

---

### Find the Best Value for k Using the PCA Data

In [19]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 12))
k

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [20]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
for i in k:
    
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters   
# 2. Fit the model to the data using `df_market_data_pca`
    kmeans_model = KMeans(n_clusters = i, random_state = 0)
    kmeans_model.fit(df_market_data_pca)
    
# 3. Append the model.inertia_ to the inertia list
    inertia.append(kmeans_model.inertia_)

In [21]:
# Create a dictionary with the data to plot the Elbow curve
pca_elbow_data = {'k': k, 'inertia': inertia}

# Create a DataFrame with the data to plot the Elbow curve
pca_elbow_data_df = pd.DataFrame(pca_elbow_data)
pca_elbow_data_df.head()

In [29]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
pca_line_chart = pca_elbow_data_df.hvplot.line(x ='k', y ='inertia', title ='PCA Elbow Curve', xticks = k)
pca_line_chart

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [30]:
# Initialise the K-Means model using the best value for k
kmeans_model = KMeans(n_clusters = 4, random_state = 0)

In [22]:
# Fit the K-Means model using the PCA data
kmeans_model.fit(df_market_data_pca)

In [23]:
# Predict the clusters to group the cryptocurrencies using the PCA data
predict = kmeans_model.predict(df_market_data_pca)

# Print the resulting array of cluster values.
predict

In [24]:
# Create a copy of the DataFrame with the PCA data
copy_pca_elbow_data_df = df_market_data_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
copy_pca_elbow_data_df['cluster'] = predict

# Display sample data
copy_pca_elbow_data_df.head()

In [34]:
# Created a scatter plot using hvPlot by setting x='PCA1' and y='PCA2'. Coloured the graph points with the labels
# found using K-Means, and added the crypto name in the 'hover_cols' parameter to identify the cryptocurrency
# represented by each data point:
pca_scatter = copy_pca_elbow_data_df.hvplot.scatter(x = 'PCA1',
                                                    y = 'PCA2',
                                                    hover_cols = 'coin_id',
                                                    title = 'PCA Scatter Plot'
                                                    )
pca_scatter

:Scatter   [PCA1]   (PCA2,coin_id)

### Visualise and Compare the Results

In this section, you will visually analyse the cluster analysis results by contrasting the outcome with and without using the optimisation techniques.

In [35]:
# Composite plot to contrast the Elbow curves
pca_line_chart + optimal_k_value

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [36]:
# Composite plot to contrast the clusters
pca_scatter + scatterplot

:Layout
   .Scatter.I  :Scatter   [PCA1]   (PCA2,coin_id)
   .Scatter.II :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

#### Answer the following question: 

  * **Question:** After visually analysing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** The inertia went down, and the data points don't seem as spread out anymore. With fewer features in play for clustering, this makes things easier to understand and see better results.
  